In [8]:
import numpy as np 
import pandas as pd
from os import listdir
import cv2
import os

import sys
module_path ='/home/Mingke.Chen/OwO/hh'



class batch_data_generater:
    def __init__(self):
        self.test_file=[]

        
    def get_file_address(self,class_path,list_names):
        image_path=[]
        for names in list_names:
            image_path.append(os.path.join(class_path,names))
        return image_path
   
    def address_collection(self,filepath,merge=True,type='train'):
        if type=='train':
            self.classes=sorted(listdir(filepath[0]))
            classes_dict={}
            for class_index in range(len(self.classes)):
                classes_dict.update({self.classes[class_index]:class_index})
            self.classes_dict=classes_dict
        
        number_of_classes=len(self.classes)
        number_of_locations=len(filepath)
        storage_file=[]
  
        for storage_file_index in range(number_of_locations):
            location_storage_file=[]
            for classes_index in range(number_of_classes):
                class_path=os.path.join(filepath[storage_file_index],self.classes[classes_index])
                #
                image_path=self.get_file_address(class_path,listdir(class_path))
                location_storage_file.append(image_path)
            storage_file.append(location_storage_file)      
        if merge == True:
            storage_file_reduce=[]
            for classes_index in range(number_of_classes):
                images_address=[]
                for storage_file_index in range(len(storage_file)):
                    images_address=images_address+storage_file[storage_file_index][classes_index]
                    
                storage_file_reduce.append(images_address)
            storage_file=np.asarray(storage_file_reduce)
        else:
            storage_file=np.asarray(storage_file)
      
        return storage_file
    
    def label_Initialization(self,storage_file,framework_format='tensorflow',merge=True):
        if framework_format=='tensorflow':
            if merge == True:
                storage_label=[]
                for class_ in range(len(storage_file)):
                    label=np.zeros([len(storage_file[class_]),len(storage_file)])
                    for index in range(len(label)):
                        label[index][class_]=1
                    storage_label.append(label)
            
            storage_label=np.asarray(storage_label)    
        else:
            raise NameError('framework_format must be tensorflow')
        return storage_label
    
    def batch_table_Initialization(self):
        #get class number
        batch_table=np.zeros(len(self.storage_file))
        
        for class_ in range(len(self.storage_file)):
            batch_table[class_]=len(self.storage_file[class_])
            
        return batch_table
    
    def max_epoch_length(self,batch_size):
    
        epochs=np.asarray(self.batch_table)/batch_size
        epochs=epochs*len(epochs)
        max_epoch_index=epochs.argmax()
        max_epoch=epochs[max_epoch_index]
        
        return int(max_epoch)
        
        
    def init_random_data_index(self,X,y):
        if type(X) is not np.dtype:
            X=np.asarray(X)
        if type(y) is not np.dtype:
            y=np.asarray(y)
     
        batch_index = np.arange(0, len(X))
        np.random.shuffle(batch_index)         
        shuf_features = X[batch_index]
        shuf_labels = y[batch_index]
        
        return shuf_features,shuf_labels
    
    def BatchGD_iter(self,X,y,batchsize=False,class_number=False):
        
        if batchsize==False:
            batchsize=self.batchsize
        start=0
        recircle=False
       
        if class_number==False: 
            data_numbers=len(X)
        else:
            data_numbers=self.batch_table[class_number]
        if batchsize > data_numbers:
             raise NameError('batchsize is out of data samples:'+str(data_numbers))
        shuf_features,shuf_labels=self.init_random_data_index(X,y)
            
        while True:
            # shuffle labels and features
            end=start+batchsize
            
            #if next batch data out of current random index
            if end >= data_numbers:
                remainder=end-data_numbers
                end=data_numbers
                recircle=True
            
            features_batch=shuf_features[start:end]
            labels_batch=shuf_labels[start:end]
          
            start=end

            #if next batch data out of current random index  init a new random index
            if recircle == True:
                recircle=False
                start=0
                shuf_features,shuf_labels=self.init_random_data_index(X,y)
                features_batch=np.append(features_batch,shuf_features[start:remainder])
                labels_batch=np.concatenate([labels_batch,shuf_labels[start:remainder]])
                start=remainder
               
            #->random->batch->init->give other
       
            yield features_batch, labels_batch

    def create_balance_batch_list_for_classes(self,balanced_batch_size,classes,batch_size):
        balanced_batch_classes_size_list=[]
        for class_ in range(classes):
            balanced_batch_classes_size_list.append(int(balanced_batch_size))
        remainder=batch_size%(balanced_batch_size*classes)
        if remainder!=0:
            for i in range(remainder):
                balanced_batch_classes_size_list[i]+=1
        
        return balanced_batch_classes_size_list
    
    def image_address_to_image(self,address):
        image=[]
        for index in range(len(address)):
            
            image_=cv2.imread(address[index])
            image_=image_/128
            image.append(image_)
        
        return np.asarray(image)
        
    
    def balance_batch_Generater(self,batch_size):
        
        classes=len(self.storage_label)
        balanced_batch_size=int(batch_size/classes)
        balanced_batch_classes_size_list=self.create_balance_batch_list_for_classes(balanced_batch_size,classes,batch_size)
         
        batch_storage=[]
        for class_ in range(classes):
            batch_storage.append(self.BatchGD_iter(self.storage_file[class_],self.storage_label[class_],
                                                   balanced_batch_classes_size_list[class_]))
        while True:
            for class_ in range(classes):
                if class_== 0:
                    batch_data,batch_label=next(batch_storage[class_])
                 
                else:
                    new_batch_data,new_batch_label=next(batch_storage[class_])
                 
                    batch_data=np.append(batch_data,new_batch_data,axis=0)
                    batch_label=np.append(batch_label,new_batch_label,axis=0)
           
            batch_data_image=self.image_address_to_image(batch_data)
            
            yield batch_data_image,batch_label
            
    
    def get_data_batch(self,batch_size,batch_type='balance',list_classes=False):
        
        if batch_type == 'balance': 
            Batch_Generater=self.balance_batch_Generater(batch_size)
            return Batch_Generater
     
        
    def filepath_Initialization(self,filepath):
        if type(filepath) is str:
            self.filepath=[filepath]
        elif type(filepath) is list:
            self.filepath=filepath
        else:
            raise NameError('type of filepath must be str or list')
        train_path_list=[]
   
        for filepath in self.filepath:
            train_path=filepath
            if not os.path.exists(train_path):
                raise NameError('No "train" directories under filepath' )
            train_path_list.append(train_path)
  
        
        self.train_path=train_path_list

    
    def get_batch(self,filepath,batch_size=100,batch_type='balance'):
        #
        self.filepath_Initialization(filepath)
        self.storage_file=self.address_collection(self.train_path)
        self.storage_label=self.label_Initialization(self.storage_file)
        self.batch_table=self.batch_table_Initialization()
        self.epoch=self.max_epoch_length(batch_size)

        
        Batch_Generater=self.get_data_batch(batch_size=batch_size,batch_type=batch_type)
        return Batch_Generater
